In [1]:
import pandas as pd
import numpy as np
import scipy
import sklearn 
import datetime as dt
import xgboost as XGB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.utils import shuffle

from sklearn import metrics
from sklearn.model_selection import train_test_split

# pd.options.display.float_format = '{:20,.0f}'.format # suprimimos la notacion cientifica en los outputs

In [25]:
path = 'data/'
features = pd.read_csv(path + 'features.csv')
test = pd.read_csv(path + 'testModified.csv')

# KNN

In [5]:
x = features.drop('precio',1)
y = features['precio']
tst_sz = 0.3 # Mejor valor
seed = 139
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=tst_sz,random_state=seed)
x, y = shuffle(x, y, random_state=seed)

In [7]:
gsc = GridSearchCV(
        estimator=KNeighborsClassifier(algorithm = 'kd_tree'),
        param_grid={'n_neighbors': [579]},
        cv=3, scoring='neg_mean_absolute_error', verbose=50, n_jobs=-1)
grid_result = gsc.fit(x, y)
best_params = grid_result.best_params_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/home/seba/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.8min finished


/home/seba/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [8]:
best_params['n_neighbors']

579

In [10]:
best_knn = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'],algorithm = 'kd_tree')
scoring = {'abs_error': 'neg_mean_absolute_error'}
scores = cross_validate(best_knn, x, y, cv=3, scoring=scoring, return_train_score=True, verbose=30, n_jobs=1)
print("MAE :", abs(scores['test_abs_error'].mean()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/seba/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


[CV]  ................................................................
[CV]  , abs_error=(train=-991545.048, test=-1062756.939), total= 1.0min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s


[CV]  , abs_error=(train=-1022578.648, test=-1002033.218), total=  52.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.7min remaining:    0.0s


[CV]  , abs_error=(train=-1033145.043, test=-990213.781), total=  53.3s
MAE : 1018334.6462888286


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.4min finished


In [26]:
best_knn.fit(x,y)
knn_predict = best_knn.predict(test.drop(['id'], axis=1))

In [27]:
res = pd.DataFrame(knn_predict, index=test.id, columns=['precio'])

In [28]:
res = res.rename({'precio':'target'}, axis=1)

In [30]:
res.to_csv("predictions/knn_prediction.csv", header = True)

In [31]:
res.shape

(60000, 1)